# Implementando uma rede simples (MLP) usando o Keras

Giovanna Furlan Torres

Turma 4 - Sistemas de Informação

## Problemática

Desenvolva um modelo sequencial em Keras com uma única camada Dense, utilizando uma unidade com a função de ativação sigmoid. Compile o modelo utilizando o otimizador adam, a função de perda binary_crossentropy, e a métrica accuracy. Inclua também a métrica F1 para uma avaliação mais completa, e explique brevemente a função de cada um desses componentes no treinamento.


Treine o modelo por 50 épocas com um batch size de 10. Após o treinamento, utilize o modelo para prever os rótulos do conjunto de teste e calcule tanto a acurácia quanto a métrica F1. Interprete os resultados, discutindo o desempenho do modelo e possíveis melhorias.




## Solução

Este projeto tem como objetivo desenvolver um modelo preditivo utilizando o dataset público do Kaggle que contém o quadro de medalhas das Olimpíadas de 1994 a 2024. O intuito é prever quais países têm maior probabilidade de estar entre os 5 melhores nas próximas competições.

A classificação dos países foi obtida a partir do site Olympics.com, que possui todo o registro dos jogos.

- Dataset : https://www.kaggle.com/datasets/youssefismail20/olympic-games-1994-2024

- Informação do quadro de medalhas: https://olympics.com/pt/olympic-games

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso.

## Bibliotecas

Essa etapa envolve a instalação de bibliotecas e configuração de outros ajustes necessários.

In [226]:
import numpy as np
import pandas as pd
import sqlite3
import gdown
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

## Importação dos dados

Para garantir o acesso aos dados por todos, sem a necessidade de permissões, foi utilizado o gdown para consumir esse dataset.

In [227]:
arquivo_destino_colab = "dataset.db"

doc_id = "1WJcaDY36SEtB4gfO4V0Pod0exq1RUJPC"

URL = f"https://drive.google.com/uc?id={doc_id}"

gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1WJcaDY36SEtB4gfO4V0Pod0exq1RUJPC
To: /content/dataset.db
100%|██████████| 65.5k/65.5k [00:00<00:00, 72.6MB/s]


'dataset.db'

O dataset está em formato ***.db***, então, após seu carregamento, foi necessário estabelecer uma conexão com o banco de dados para extrair as tabelas. A lista de tabelas foi obtida no Kaggle. Em seguida, todos os dados foram salvos em um dataframe unificado e exportados para um arquivo .csv.

In [228]:
# Conexão com o DB SQLite
conn = sqlite3.connect(arquivo_destino_colab)

# Lista com os nomes das tabelas
tables = [
    'SaltLakeCity_2002_Olympics_Nations_Medals',
    'Sochi_2014_Olympics_Nations_Medals',
    'Vancouver_2010_Olympics_Nations_Medals',
    'London_2012_Olympics_Nations_Medals',
    'Sydney_2000_Olympics_Nations_Medals',
    'Rio_2016_Olympics_Nations_Medals',
    'Paris_2024_Olympics_Nations_Medals',
    'Athens_2004_Olympics_Nations_Medals',
    'Atlanta_1996_Olympics_Nations_Medals',
    'Tokyo_2020_Olympics_Nations_Medals',
    'Torino_2006_Olympics_Nations_Medals',
    'PyeongChang_2018_Olympics_Nations_Medals',
    'Nagano_1998_Olympics_Nations_Medals',
    'Lillehammer_1994_Olympics_Nations_Medals',
    'beijing_2022_Olympics_Nations_Medals'
]

dfs = []
for table in tables:

    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    # Extrair o nome da cidade e ano da tabela para criar a coluna 'City_Year'
    city_year = table.split('_Olympics_Nations_Medals')[0].replace('_', ' ')
    df['City_Year'] = city_year
    dfs.append(df)

# Concatenar todos os DataFrames em um único DataFrame
dados = pd.concat(dfs, ignore_index=True)


# Fechar a conexão com o DB e salvar o DF em um CSV
conn.close()
dados.to_csv('/content/drive/MyDrive/M11/Unified_Olympics_Medals.csv', index=False)


Exibição dos dados após o carregamento e a identificação do Ano e Cidade em que ocorreu os jogos.

In [229]:
dados

,NOC,Gold,Silver,Bronze,Total,City_Year
0,NOR,13,5,7,25,SaltLakeCity 2002
1,GER,12,16,8,36,SaltLakeCity 2002
2,USA,10,13,11,34,SaltLakeCity 2002
3,CAN,7,3,7,17,SaltLakeCity 2002
4,RUS,5,4,4,13,SaltLakeCity 2002
...,...,...,...,...,...,...
787,UKR,0,1,0,1,beijing 2022
788,BEL,0,1,0,1,beijing 2022
789,EST,0,0,1,1,beijing 2022
790,LVA,0,0,1,1,beijing 2022


### **Descrição das colunas do Dataset**

- **NOC:** Código do Comitê Olímpico Nacional (identifica o país).

- **Gold:** Número de medalhas de ouro ganhas pelo país.

- **Silver:** Número de medalhas de prata.

- **Bronze:** Número de medalhas de bronze.

- **Total:** Número total de medalhas do país naquela edição.

- **City_Year:** Cidade e ano dos Jogos Olímpicos.

### **Informações Gerais do Dataset analisado**

In [230]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   NOC        792 non-null    object
 1   Gold       792 non-null    int64 
 2   Silver     792 non-null    int64 
 3   Bronze     792 non-null    int64 
 4   Total      792 non-null    int64 
 5   City_Year  792 non-null    object
dtypes: int64(4), object(2)
memory usage: 37.2+ KB


### **Sumário Estatístico**

Abaixo, apresenta-se uma visão geral dos dados olímpicos:

- **Número de amostras:** Indica a quantidade total de registros no conjunto de dados.

- **Número de features:** Mostra o total de variáveis presentes em cada amostra.

- **Tipos de dados das features:** Exibe os tipos de dados associados a cada coluna do DataFrame.

- **Descrição estatística das features numéricas:** Fornece estatísticas descritivas como média, desvio padrão, valores mínimo e máximo, entre outros, para as variáveis numéricas.

- **Valores ausentes por coluna:** Lista a quantidade de valores ausentes em cada coluna do DataFrame.

- **Valores únicos em cada coluna:** Apresenta o número de valores distintos encontrados em cada coluna.


In [231]:
num_amostras, num_features = dados.shape
print(f"Número de amostras: {num_amostras}")
print(f"Número de features: {num_features}")

print("\nTipos de dados das features:")
print(dados.dtypes)

print("\nDescrição estatística das features numéricas:")
print(dados.describe())

print("\nValores ausentes por coluna:")
print(dados.isnull().sum())

print("\nValores únicos em cada coluna:")
print(dados.nunique())

Número de amostras: 792
Número de features: 6

Tipos de dados das features:
NOC          object
Gold          int64
Silver        int64
Bronze        int64
Total         int64
City_Year    object
dtype: object

Descrição estatística das features numéricas:
             Gold      Silver      Bronze       Total
count  792.000000  792.000000  792.000000  792.000000
mean     3.582071    3.584596    3.950758   11.117424
std      6.294768    5.542427    5.516906   16.593911
min      0.000000    0.000000    0.000000    1.000000
25%      0.000000    1.000000    1.000000    2.000000
50%      1.000000    2.000000    2.000000    5.000000
75%      4.000000    4.000000    5.000000   13.000000
max     48.000000   44.000000   42.000000  126.000000

Valores ausentes por coluna:
NOC          0
Gold         0
Silver       0
Bronze       0
Total        0
City_Year    0
dtype: int64

Valores únicos em cada coluna:
NOC          136
Gold          35
Silver        35
Bronze        35
Total         66
City_Ye

# Preparação para implementação do modelo

O dataset `dados` é composto por dados dos Jogos Olímpicos de 1994 a 2024, incluindo informações sobre o número de medalhas de ouro, prata e bronze conquistadas por cada país em diferentes edições. O objetivo principal é utilizar esses dados para construir um modelo de classificação que determine se um país tem chances de ficar entre as cinco primeiras posições no quadro geral de medalhas em uma determinada edição dos Jogos.





## **Variável Alvo**

A variável alvo no modelo de classificação é chamada de `Top5`. Ela é uma variável binária que indica se o país ficou ou não entre os cinco primeiros no quadro geral de medalhas, com as seguintes categorias:

- **"Sim"**: Indica que o país ficou entre os cinco primeiros no total de medalhas (soma das medalhas de ouro, prata e bronze).
- **"Não"**: Indica que o país não ficou entre os três primeiros.

## Criação da coluna TOP 5

Com base nas informações disponíveis no site olympics.com, foi possível obter os dados necessários para completar o dataset utilizado no modelo. Foram listadas as edições dos Jogos Olímpicos e identificados os 5 primeiros colocados em cada uma delas.

In [232]:
# Dicionário com as edições dos Jogos e os países que ficaram entre os 5 primeiros colocados
top5 = {
    'Lillehammer 1994': {'NOR', 'GER', 'RUS', 'ITA', 'USA'},
    'Atlanta 1996': {'USA', 'GER', 'RUS', 'CHN', 'AUT'},
    'Nagano 1998': {'GER', 'NOR', 'RUS', 'AUT', 'CAN'},
    'Sydney 2000': {'USA', 'RUS', 'CHN', 'AUS', 'GER'},
    'SaltLakeCity 2002': {'GER', 'USA', 'NOR', 'CAN', 'AUT'},
    'Athens 2004': {'USA', 'RUS', 'CHN', 'AUS', 'GER'},
    'Torino 2006': {'GER', 'USA', 'CAN', 'AUT', 'RUS'},
    'Vancouver 2010': {'USA', 'GER', 'CAN', 'NOR', 'AUT'},
    'London 2012': {'USA', 'CHN', 'GBR', 'RUS', 'GER'},
    'Sochi 2014': {'RUS', 'USA', 'NOR', 'CAN', 'NED'},
    'Rio 2016': {'USA', 'CHN', 'GBR', 'RUS', 'GER'},
    'PyeongChang 2018': {'NOR', 'GER', 'CAN', 'USA', 'NED'},
    'Tokyo 2020': {'USA', 'CHN', 'ROC', 'GBR', 'JPN'},
    'beijing 2022': {'NOR', 'ROC', 'GER', 'CAN', 'USA'},
    'Paris 2024': {'USA', 'CHN', 'JPN', 'AUS', 'FRA'}
}

# Cria uma nova coluna 'Top5' no DataFrame 'dados', onde cada linha indica se o país ('NOC') está entre os 5 primeiros colocados
dados['Top5'] = dados.apply(
    lambda row: 'Sim' if row['NOC'] in top5.get(row['City_Year'], set()) else 'Não',
    axis=1
)

# Retorna o DataFrame atualizado com a nova coluna 'Top5'
dados


,NOC,Gold,Silver,Bronze,Total,City_Year,Top5
0,NOR,13,5,7,25,SaltLakeCity 2002,Sim
1,GER,12,16,8,36,SaltLakeCity 2002,Sim
2,USA,10,13,11,34,SaltLakeCity 2002,Sim
3,CAN,7,3,7,17,SaltLakeCity 2002,Sim
4,RUS,5,4,4,13,SaltLakeCity 2002,Não
...,...,...,...,...,...,...,...
787,UKR,0,1,0,1,beijing 2022,Não
788,BEL,0,1,0,1,beijing 2022,Não
789,EST,0,0,1,1,beijing 2022,Não
790,LVA,0,0,1,1,beijing 2022,Não


##  Padronização

A padronização é um processo de transformação dos dados em que as características numéricas são ajustadas para terem uma média de 0 e um desvio padrão de 1. Isso garante que todas as variáveis contribuam de maneira equilibrada para o modelo, evitando que variáveis com valores maiores dominem as menores. No código a seguir, padronizou-se as colunas numéricas de medalhas (ouro, prata, bronze e total) do dataset.

In [233]:
# Lista com as colunas numéricas
numeric_columns = ['Gold', 'Silver', 'Bronze', 'Total']

# Cria uma instância para padronizar os dados (média = 0 e desvio padrão = 1)
scaler = StandardScaler()

# Aplica a padronização e substitui os valores originais pelas versões padronizadas
dados[numeric_columns] = scaler.fit_transform(dados[numeric_columns])

# Retorna o DataFrame atualizado
dados

,NOC,Gold,Silver,Bronze,Total,City_Year,Top5
0,NOR,1.497097,0.255538,0.553058,0.837135,SaltLakeCity 2002,Sim
1,GER,1.338135,2.241482,0.734434,1.500448,SaltLakeCity 2002,Sim
2,USA,1.020210,1.699861,1.278560,1.379846,SaltLakeCity 2002,Sim
3,CAN,0.543323,-0.105543,0.553058,0.354726,SaltLakeCity 2002,Sim
4,RUS,0.225398,0.074997,0.008931,0.113521,SaltLakeCity 2002,Não
...,...,...,...,...,...,...,...
787,UKR,-0.569415,-0.466624,-0.716571,-0.610092,beijing 2022,Não
788,BEL,-0.569415,-0.466624,-0.716571,-0.610092,beijing 2022,Não
789,EST,-0.569415,-0.647164,-0.535195,-0.610092,beijing 2022,Não
790,LVA,-0.569415,-0.647164,-0.535195,-0.610092,beijing 2022,Não


## Codificação de variáveis categóricas

O `LabelEncoder` é uma ferramenta usada para converter dados categóricos em valores numéricos. Ele atribui um número inteiro único a cada categoria. Por exemplo, a coluna `NOC` contém países como "USA", "CHN", "RUS", o `LabelEncoder` converterá essas categorias em números como 0, 1, 2, permitindo que o modelo de Machine Learning processe essas informações.

In [234]:
label_encoder = LabelEncoder()  # Cria uma instância do codificador de rótulos
dados['NOC'] = label_encoder.fit_transform(dados['NOC'])  # Converte a categoria (NOC) em valores numéricos
dados['NOC']

,NOC
0,89
1,47
2,129
3,20
4,106
...,...
787,127
788,12
789,39
790,76


## Ajuste para classificação binária

Neste passo, a variável Top5, que é a variável alvo do modelo de classificação binária, será convertida para valores numéricos. A padronização envolve mapear as respostas "Sim" e "Não" para valores binários 1 e 0, respectivamente.

In [235]:
dados['Top5'] = dados['Top5'].map({'Sim': 1, 'Não': 0})

# Aplicação do Modelo

## Divisão dos dados

Aqui, os dados são separados em variáveis independentes (X) e variável dependente ou alvo (y). Em seguida, os dados são divididos em conjuntos de treino e teste. O conjunto de treino é usado para ajustar o modelo, enquanto o conjunto de teste é utilizado para avaliar a performance do modelo em dados não vistos.

In [236]:
# Selecionar as variáveis independentes
X = dados[['NOC', 'Gold', 'Silver', 'Bronze', 'Total']]

# Selecionar a variável dependente
y = dados['Top5']

# Dividir em treino e teste (80% - 20%, respectivamente)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Exibir as dimensões dos conjuntos de treino e teste
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (633, 5)
X_test shape: (159, 5)
y_train shape: (633,)
y_test shape: (159,)


## Métricas

Esta função calcula a métrica F1 e a acurácia, duas métricas importantes para avaliar a performance de modelos de classificação binária:

- **Métrica F1**: A métrica F1 é a média harmônica da precisão (precision) e da sensibilidade (recall). Ela fornece um único valor que equilibra esses dois aspectos e é especialmente útil quando os dados estão desequilibrados, ou seja, quando uma das classes é muito mais comum do que a outra.

- **Acurácia**: A acurácia é a proporção de previsões corretas em relação ao número total de previsões. Ela mede a capacidade geral do modelo de classificar corretamente todas as instâncias, sendo uma métrica direta e intuitiva.



> **OBSERVAÇÃO:** Essas métricas são usadas na compilação do modelo com `metrics=['accuracy', 'f1_score']` para monitorar o desempenho durante o treinamento. A inclusão das métricas na compilação permite que sejam avaliadas automaticamente ao longo do processo de treinamento e validação do modelo, por isso foram adicionadas nessa etapa do código, seus resultados serão apresentados posteriormente.


In [237]:
def metricas(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')  # Garantir que y_true é float32
    y_pred = K.cast(y_pred, 'float32')  # Garantir que y_pred é float32

    # Arredondar os valores de y_true e y_pred para 0 ou 1
    y_true = K.round(y_true)
    y_pred = K.round(y_pred)

    # Calcular os verdadeiros positivos (tp)
    tp = K.sum(K.cast(y_true * y_pred, 'float32'), axis=0)

    # Calcular os falsos positivos (fp)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float32'), axis=0)

    # Calcular os falsos negativos (fn)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float32'), axis=0)

    # Calcular a precisão (precision)
    precision = tp / (tp + fp + K.epsilon())

    # Calcular a sensibilidade (recall)
    recall = tp / (tp + fn + K.epsilon())

    # Calcular a métrica F1
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    # Calcular a acurácia
    accuracy = K.mean(K.cast(K.equal(y_true, y_pred), 'float32'))

    # Retornar a métrica F1 e a acurácia
    return f1, accuracy

## Construção do modelo

Este trecho de código constrói e compila um modelo de classificação binária utilizando a biblioteca Keras. O modelo é simples, composto por uma única camada densa (Dense) com uma unidade e uma função de ativação sigmoide (sigmoid).

Essa configuração é apropriada para problemas de classificação binária, onde a saída é um valor entre 0 e 1, representando a probabilidade de uma classe.

In [238]:
# Criação do modelo
model = Sequential()
model.add(Dense(units=1, activation='sigmoid', input_shape=(X_train.shape[1],)))

# Compilação do modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'f1_score']
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



## Treinamento do modelo

O treinamento é configurado para ocorrer 50 épocas, com um tamanho de batch de 10. Durante o processo, o modelo também é validado periodicamente utilizando o conjunto de dados de teste, permitindo avaliar o desempenho em dados não vistos. A função fit retorna um histórico (history) que contém informações sobre o desempenho do modelo ao longo das épocas, como a perda e as métricas de avaliação.

In [239]:
# Treinamento do modelo
history = model.fit(
    X_train,  # Dados de entrada para treinamento
    y_train,  # Labels de treinamento
    epochs=50,  # Número de épocas
    batch_size=10,  # Tamanho do batch
    validation_data=(X_test, y_test),  # Dados de validação
    verbose=1  # Progresso do treinamento
)

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0904 - f1_score: 0.1601 - loss: 27.5184 - val_accuracy: 0.1006 - val_f1_score: 0.1724 - val_loss: 24.0585
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0944 - f1_score: 0.1610 - loss: 23.9360 - val_accuracy: 0.1006 - val_f1_score: 0.1724 - val_loss: 20.3629
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1032 - f1_score: 0.1716 - loss: 20.5498 - val_accuracy: 0.1069 - val_f1_score: 0.1724 - val_loss: 16.6669
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1100 - f1_score: 0.1676 - loss: 15.9657 - val_accuracy: 0.1195 - val_f1_score: 0.1724 - val_loss: 13.0364
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1389 - f1_score: 0.1800 - loss: 11.9580 - val_accuracy: 0.1447 - val_f1_score: 0.1724 - val_loss: 9.3770
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1856 - f1_score: 0.2241 - loss: 8.1651 - val_accuracy: 0.1824 - val_f1_score: 0.1724 - 

## Resultado das métricas

Os comandos a seguir exibem os resultados das métricas de avaliação do modelo:

In [240]:
print(f'Acurácia: {accuracy:.4f}')
print(f'Métrica F1: {f1:.4f}')

Acurácia: 0.9371
Métrica F1: 0.6486


Análise dos resultados obtidos:

- **Acurácia: 0.9371** indica que o modelo acertou aproximadamente 93.71% das classificações feitas, considerando todas as classes.

- **Métrica F1: 0.6486** reflete o equilíbrio entre a classificação e a sensibilidade do modelo. Embora a acurácia seja alta, a métrica F1 mais baixa sugere que o modelo pode ter um desempenho desigual na identificação das classes positivas e negativas.

# Classificação do modelo

O código a seguir realiza previsões usando o modelo treinado e gera uma tabela que indica se cada país está previsto para estar entre os cinco primeiros colocados.

In [241]:
# Fazer previsões no conjunto completo
y_pred_prob_full = model.predict(X)
predictions_df = dados[['NOC']].copy()
predictions_df['Predicted_Prob'] = y_pred_prob_full

# Decodificar os valores numéricos de NOC para os rótulos originais
predictions_df['NOC'] = label_encoder.inverse_transform(predictions_df['NOC'])

# Agrupar por NOC e selecionar a maior probabilidade de cada grupo
predictions_df = predictions_df.groupby('NOC').agg({'Predicted_Prob': 'max'}).reset_index()
predictions_df = predictions_df.sort_values(by='Predicted_Prob', ascending=False)

# Identificar os 5 países com maior probabilidade
top_5_countries = predictions_df.head(5)['NOC'].tolist()
predictions_df['In_Top5'] = np.where(predictions_df['NOC'].isin(top_5_countries), 'Sim', 'Não')

# Criar a tabela final com as colunas necessárias
resultados = predictions_df[['NOC', 'In_Top5']]
print("Classificação dos países que poderão estar no top 5:")
resultados


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Classificação dos países que poderão estar no top 5:


,NOC,In_Top5
129,USA,Sim
22,CHN,Sim
106,RUS,Sim
45,GBR,Sim
6,AUS,Sim
...,...,...
125,UAE,Não
132,VIE,Não
135,ZIM,Não
128,URU,Não


Com base na classificação do modelo, os países que estariam no top 5 do quadro de medalhas das Olimpíadas são:

- **CHN** (China)
- **USA** (Estados Unidos)
- **RUS** (Rússia)
- **GBR** (Reino Unido)
- **AUS** (Austrália)

# Visualizações Gráficas

## Matriz de confusão

A matriz de confusão é uma ferramenta usada para avaliar a performance de um modelo. Ela mostra a contagem de previsões corretas e incorretas em cada classe, permitindo visualizar como o modelo está classificando as instâncias em comparação com os rótulos reais.

A matriz de confusão é estruturada da seguinte forma:

- **Verdadeiros Positivos (VP)**: Casos em que o modelo previu corretamente a classe positiva.
- **Falsos Positivos (FP)**: Casos em que o modelo previu a classe positiva, mas o rótulo real era negativo.
- **Verdadeiros Negativos (VN)**: Casos em que o modelo previu corretamente a classe negativa.
- **Falsos Negativos (FN)**: Casos em que o modelo previu a classe negativa, mas o rótulo real era positivo.

Abaixo está a matriz de confusão gerada para o modelo, mostrando a distribuição das previsões do modelo em comparação com os rótulos reais.

In [242]:
# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Criar o gráfico da matriz de confusão
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Não', 'Sim'],
    y=['Não', 'Sim'],
    colorscale='Greens',
    colorbar=dict(title='Contagem'),
    zmin=0,
    text=cm,
    texttemplate='%{text}',
    textfont=dict(size=12, color='#8B8B83')
))

# Atualizar layout
fig_cm.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predito',
    yaxis_title='Real',
    xaxis=dict(tickvals=[0, 1], ticktext=['Não', 'Sim']),
    yaxis=dict(tickvals=[0, 1], ticktext=['Não', 'Sim']),
)

# Exibe o gráfico
fig_cm.show()

O modelo apresenta uma boa precisão, indicando que suas previsões positivas são, na maioria das vezes, corretas. No entanto, com base nos que ele classificou incorretamente, ainda possui espaço para melhorias.

## Análise do treinamento

A seguir, serão apresentados três gráficos que ajudam a visualizar o desempenho do modelo durante o treinamento:

1. **Perda durante o treinamento:** Mostra como a perda do modelo evolui ao longo das épocas, tanto para os dados de treinamento quanto para validação. Isso ajuda a entender se o modelo está aprendendo e ajustando seus parâmetros corretamente.

2. **Acurácia durante o treinamento:** Exibe a acurácia do modelo ao longo das épocas para os dados de treinamento e validação. Esse gráfico indica a capacidade do modelo de fazer previsões corretas ao longo do tempo.

3. **Métrica F1 durante o treinamento:** Mostra a métrica F1 ao longo das épocas, para treinamento e validação. A métrica F1 é importante para avaliar o equilíbrio entre precisão e recall, especialmente em conjuntos de dados desbalanceados.

In [243]:
# Plotar a perda de treinamento e validação
fig_loss = go.Figure()

fig_loss.add_trace(go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines',
    name='Treinamento Loss'
))

fig_loss.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines',
    name='Validação Loss'
))

fig_loss.update_layout(
    title='Perda durante o treinamento',
    xaxis_title='Épocas',
    yaxis_title='Loss',
    legend_title='Legenda'
)

# Plotar a acurácia de treinamento e validação
fig_accuracy = go.Figure()

fig_accuracy.add_trace(go.Scatter(
    x=list(range(1, len(history.history['accuracy']) + 1)),
    y=history.history['accuracy'],
    mode='lines',
    name='Treinamento Acurácia'
))

fig_accuracy.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_accuracy']) + 1)),
    y=history.history['val_accuracy'],
    mode='lines',
    name='Validação Acurácia'
))

fig_accuracy.update_layout(
    title='Acurácia durante o treinamento',
    xaxis_title='Épocas',
    yaxis_title='Acurácia',
    legend_title='Legenda'
)

# Plotar a métrica F1 durante o treinamento
fig_f1 = go.Figure()

fig_f1.add_trace(go.Scatter(
    x=list(range(1, len(history.history['f1_score']) + 1)),
    y=history.history['f1_score'],
    mode='lines',
    name='Treinamento F1'
))

fig_f1.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_f1_score']) + 1)),
    y=history.history['val_f1_score'],
    mode='lines',
    name='Validação F1'
))

fig_f1.update_layout(
    title='F1 Score durante o treinamento',
    xaxis_title='Épocas',
    yaxis_title='F1 Score',
    legend_title='Legenda'
)

# Exibir os gráficos
fig_loss.show()
fig_accuracy.show()
fig_f1.show()


### Análise Geral dos gráficos


1. **Perda durante o Treinamento:**
   - Neste gráfico, a loss diminui à medida que o modelo é treinado. A linha de treinamento cai rapidamente, enquanto a linha de validação também diminui, mas em um ritmo mais lento. Isso sugere que o modelo está aprendendo bem e não está sofrendo de overfitting.

2. **Acurácia durante o Treinamento:**
   - Neste gráfico, a acurácia aumenta à medida que o modelo é treinado. A linha de treinamento mostra uma melhoria constante, assim como a linha de validação, embora em um ritmo mais lento.

   - Isso indica indica que o modelo está aprendendo efetivamente e não está

3. **F1 Score durante o Treinamento:**
   - O gráfico mostra que o F1 Score permanece constante tanto no treinamento quanto na validação. Isso pode indicar que o modelo está alcançando um equilíbrio entre precisão e recall, mas é importante verificar pois as classes estão em quantidades desbalanceadas.


# Conclusão

Para concluir, há várias melhorias que podem ser aplicadas ao modelo. Primeiramente, o rebalanceamento dos dados, podendo ser feito por meio de sobreamostragem da classe minoritária ("SIM") ou subamostragem da classe majoritária ("NÃO"), o que ajudaria a equilibrar melhor as classes e a aumentar o F1 Score. Além disso, o ajuste de hiperparâmetros, como testar diferentes arquiteturas de rede neural e modificar o número de unidades na camada densa, pode otimizar o desempenho do modelo.

Outra possibilidade é a avaliação com métricas adicionais, como a curva ROC e a métrica AUC, para uma visão mais aprofundada da capacidade do modelo de distinguir entre as classes.
